In [103]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
import requests

In [104]:
headers = {
    'x-devtools-emulate-network-conditions-client-id': "c60c3ba6-26e8-45ce-9f9f-0b5521437cf6",
    'upgrade-insecure-requests': "1",
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
    'accept': "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    'referer': "http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247",
    'accept-encoding': "gzip, deflate, sdch",
    'accept-language': "es-ES,es;q=0.8",
    'cookie': "LANGUE_LOGIN=fr",
    'cache-control': "no-cache"
}

data = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247", headers=headers)

soup = BeautifulSoup(data.content, 'html.parser')
queries = soup.find_all('select')
queries_map = {}
for s in queries:
    queries_map[s.get('name')] = {}
    map = queries_map[s.get('name')]
    options = s.find_all('option')
    for o in options:
        if not o.string:
            value = ""
        else:
            value = o.string
        map[value] = o.get('value')

print(queries_map)

{'ww_x_PERIODE_PEDAGO': {'': 'null', 'Bachelor semestre 1': '249108', 'Bachelor semestre 6': '942175', 'Semestre automne': '953159', 'Master semestre 1': '2230106', 'Stage printemps 4ème année': '2226626', 'Stage automne 4ème année': '2226616', 'Master semestre 4': '2230140', 'Semestre printemps': '2754553', 'Projet Master automne': '249127', 'Bachelor semestre 5b': '2226768', 'Bachelor semestre 2': '249114', 'Mineur semestre 2': '2335676', 'Bachelor semestre 5': '942120', 'Bachelor semestre 3': '942155', 'Bachelor semestre 4': '942163', 'Bachelor semestre 6b': '2226785', 'Projet Master printemps': '3781783', 'Master semestre 2': '942192', 'Stage printemps master': '2227132', 'Stage automne 3ème année': '953137', 'Mineur semestre 1': '2335667', 'Stage printemps 3ème année': '983606', 'Mise à niveau': '2063602308', 'Master semestre 3': '2230128'}, 'ww_x_HIVERETE': {'': 'null', 'Semestre de printemps': '2936295', "Semestre d'automne": '2936286'}, 'ww_x_UNITE_ACAD': {'': 'null', 'Mathémat

In [154]:
def getData(unite_acad, periode_acad, periode_pedago, semester_type):
    payload = {'ww_x_GPS': 71297569,
               'ww_i_reportModel':133685247, 
               'ww_i_reportModelXsl': 133685271,
               'ww_x_UNITE_ACAD': unite_acad,
               'ww_x_PERIODE_ACAD': periode_acad,
               'ww_x_PERIODE_PEDAGO': periode_pedago,
               'ww_x_HIVERETE': semester_type
              }
    r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.XLS", params=payload)
    return r.text

In [177]:
df_final = None
df_year = None

def addData(unite_acad, periode_acad, periode_pedago, semester_type, list_semesters):
    unite_acad_id = queries_map['ww_x_UNITE_ACAD'][unite_acad]
    periode_acad_id = queries_map['ww_x_PERIODE_ACAD'][periode_acad]
    periode_pedago_id = queries_map['ww_x_PERIODE_PEDAGO'][periode_pedago]
    semester_type_id = queries_map['ww_x_HIVERETE'][semester_type]

    
    soup = BeautifulSoup(getData(unite_acad_id,periode_acad_id,periode_pedago_id,semester_type_id), 'html.parser')
    rows = soup.find_all('tr')
    print(rows)
    header_cols = rows[2].find_all('th')
    col_titles, data = [], {}
    for c in header_cols:
        data[c.string] = []
        col_titles.append(c.string)
    [col_titles.append(semestre[9:]) for semestre in list_semesters]
    for semestre in list_semesters:
        data[semestre[9:]] = [ None ] * ( len(rows) - 3)
    df = pd.DataFrame(columns=col_titles)
    global df_final, df_year
    if df_final is None:
        df_final = df.copy()
    if df_year is None:
        df_year = df.copy()
    
    
    for h in range(3, len(rows)):
        values = rows[h].find_all('td')
        for v in range(len(values)-2):
            data[header_cols[v].string].append(values[v].string)
        data[periode_pedago[9:]] = int(periode_acad[:-5])
    df = pd.DataFrame.from_dict(data)
    return df



In [178]:
df_final = None
df_year = None
unite_acad = "Informatique"
periode_acad = list(queries_map['ww_x_PERIODE_ACAD'].keys())
periode_acad.remove('')
periode_pedago = []
for p in list(queries_map['ww_x_PERIODE_PEDAGO'].keys()):
    if 'Bachelor' in p and "b" not in p:
        periode_pedago.append(p)
for i in range(4,5):
    a = addData("Informatique", "2008-2009", "Bachelor semestre " + str(i), "", sorted(periode_pedago))
    df_year = a.combine_first(df_year)
df_year

[<tr><th colspan="10">
    Ces listes d'étudiants ne seront définitives qu'ŕ la mi-aoűt une fois la commission d'admission bachelor et la conférence d'examen passées afin de s'assurer que tous les étudiants nouvellement admis ŕ l'EPFL y figurent et que les étudiants en situation d'échec définitif soient rayés de ces listes
   </th></tr>]


IndexError: list index out of range